In [ ]:
from b2d.testbed.backend.jupyter import JupyterGui
import ipywidgets
import ipycanvas
import b2d
import random

# Newtons Cradle
Here we try to implement 
[newtons cradle](https://en.wikipedia.org/wiki/Newton%27s_cradle) in Box2D
The source code for the example is in `plot_newtons_cradle.py`

![Image](https://upload.wikimedia.org/wikipedia/commons/d/d3/Newtons_cradle_animation_book_2.gif)

In [ ]:
from plot_newtons_cradle import NewtonsCradle  as Example
s = JupyterGui.Settings(resolution=[800,300])
Example.run(JupyterGui, gui_settings=s)

<p style=text-align: center>
    <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Angry_Birds_logo_2015.svg/2880px-Angry_Birds_logo_2015.svg.png width=30%>
</p>
The following shows an example simmilar to the game "Angry Birds"
   

In [ ]:
from plot_angry_shapes import AngryShapes  as Example
settings = JupyterGui.Settings(fps=30, scale=2, translate=(20,100), resolution=(1000,500))
Example.run(JupyterGui,gui_settings=settings)

# Rockets
Fly a rocket from one plant to another.
Use 'a','w','d' to fire the rockets engines,
but avoid the black hole!

In [ ]:
from plot_rocket import Rocket  as Example
settings = JupyterGui.Settings(scale=3)
Example.run(JupyterGui, gui_settings=settings)

# Gauss Machine
This example shows how to get a gaussian distribution from an uniform one with a mechanical decivce.
Note that the particles have a lifetime of 25 seconds, so the bins will at some point not get fuller (but also your RAM will not get fuller ;) )

In [ ]:
from plot_gauss_machine import GaussMachine  as Example
Example.run(JupyterGui)

# Color Mixing
This example show the color mixing capabilities of liquidfun

In [ ]:
from plot_color_mixing import ColorMixing  as Example
Example.run(JupyterGui)

# Function Shape
This exanmple shows how to draw a chain shape in the shape of a function

In [ ]:
from plot_function_shape import FunctionShape  as Example
Example.run(JupyterGui)

# World of Goo Clone
This exanmple shows how to implement a worlf of goo clone with pyb2d

In [ ]:
from plot_goo import Goo  as Example
Example.run(JupyterGui)

# Live Interaction from Cells
Here we define a testworld and interact from the cells below with the world

In [ ]:
class LiveInteraction(b2d.testbed.TestbedBase):

    name = "live interaction"
    
    def __init__(self, settings=None): 
        super(LiveInteraction, self).__init__(settings=settings)
        dimensions = [30,30]
        
        # the outer box
        box_shape = b2d.ChainShape()
        box_shape.create_loop([
                (0,0),
                (0,dimensions[1]),
             (dimensions[0],dimensions[1]),
                (dimensions[0],0)
            ]
        )
        box = self.world.create_static_body( position=(0, 0), 
            fixtures=b2d.fixture_def(
                shape = box_shape,
                friction=0
            )
        )
        self.circle = self.world.create_dynamic_body(
            position = (10 + random.random() ,10),
            fixtures = b2d.fixture_def(
                shape = b2d.circle_shape(radius=1),
                density=1
            )
        )

ui = LiveInteraction.run(JupyterGui)

In [ ]:
circles = []

In [ ]:
# insert a bodies into the running world
import time


world = ui.testbed.world
for i in range(20):
    
    with ui.world_lock:

        circle = world.create_dynamic_body(
            position = (10 + random.random() ,10),
            fixtures = b2d.fixture_def(
                shape = b2d.circle_shape(radius=1),
                density=1
            )
        )
    time.sleep(0.1)
    circles.append(circle)


In [ ]:
# apply ian impulse to each second body
for i in range(10):
    with ui.world_lock:
        for i in range(0,len(circles),2):
            circles[i].apply_linear_impulse_to_center((100,10))
    time.sleep(1)

In [ ]:
# experimental joystick support
# this may not work properly
from ipywidgets import Controller
c = Controller()

def cb(change):
    values = [a.value * 200.0 for a in change['owner'].axes]
    if len(values) >=2:
        values[1] *= -1
        circle = circles[0]
        with ui.world_lock:
            circle.apply_force_to_center(tuple(values[0:2]), True)
c.observe(cb)
c